# 단일 표본 t검정

[문제]
- 성인 여성 25명의 키 데이터
- 평균, 표준편차 계산
- ks-정규성 검정 실행
- 임의의 어떤 값 163은 평균 값에 근사할까?

In [5]:
# data 로딩
with open("./datas2/성인여성_키_데이터.txt", "r") as f:
    data = f.read().split("\n")
    data = map(float, data) #리스트 요소가 문자열 -> float로 바꿈
    data = list(data)
    print(data) # map 함수의 결과를 list로 형변화 ->출력
    print(len(data))

[150.27, 142.94, 160.99, 157.48, 153.46, 137.5, 154.94, 159.51, 171.87, 143.69, 153.65, 160.25, 153.07, 154.42, 141.21, 154.17, 162.56, 164.44, 172.36, 141.51, 169.4, 167.97, 170.26, 157.8, 167.61]
25


- 데이터의 평균과 표준편차 구하기

In [49]:
import numpy as np
# 평균
mean_value = np.mean(data)
# 표준편차
std_dev = np.std(data, ddof=0) #모집단의 표준편차
mean_value, std_dev

(np.float64(156.9332), np.float64(9.974187774450611))

## 가설 설정
[문제]

- 귀무가설: 표본 데이터의 평균은 163과 같다.
- 대립가설: 표본 데이터의 평균은 163과 다르다

## 정규성 검정(ks-test)

In [53]:
from scipy.stats import * #stats 모듈의 모든 내용을 로딩
# from scipy import stats
import numpy as np

In [54]:
# 정규성 검정 함수
#  KS-test는 주어진 표본 데이터가 특정 이론적인 분포(예: 정규분포)를 따르는지 검정하는 방법
# 'norm': 비교 대상이 되는 이론적인 분포를 지정 ('norm'은 정규분포)
# args=(np.mean(data), np.std(data, ddof=0)): 표본 데이터의 평균과 표준편차를 사용하여 정규분포의 모수 설정
ks_stats, p_value = kstest(data, 'norm', args=(mean_value, std_dev))
ks_stats, p_value

(np.float64(0.10925974986314058), np.float64(0.8950958424230889))

## 검정 하기

In [8]:
ttest_1samp(data, 163)

TtestResult(statistic=np.float64(-2.979804412662668), pvalue=np.float64(0.006510445335847954), df=np.int64(24))

# 독립 표본 t 검정
- 두 그룹 A, B의 차이를 검증할 때 사용하는 검증 방법

#문제 : A, B반의 점수 차이가 있는지 검증하기
# [가설 설정]
#H0 : A반과 B반의 평균은 같다.
#H1 : A반과 B반의 평균은 다르다.


In [14]:
import pandas as pd
import numpy as np

In [18]:
df1= pd.read_csv("./datas2/반별_점수_type1.csv" , encoding="euc-kr")
df1. tail()

,반,점수
25,B,77
26,B,75
27,B,65
28,B,61
29,B,55


In [21]:
#데이터프레임에서 반 컬럼의 유니크한 값의 갯수 확인
df1['반'].value_counts()

반
A    20
B    10
Name: count, dtype: int64

### A반과 B반 데이터 분리하기

In [25]:
#A반의 점수만 추출
groupA = df1['점수'].loc[df1['반']=='A'].values
len(groupA)

20

In [27]:
#A반의 점수만 추출
groupB = df1['점수'].loc[df1['반']=='B'].values
len(groupB)

10

In [32]:
meanA = np.mean(groupA)
stdA = np.std(groupA, ddof=1)
meanA, stdA

(np.float64(70.55), np.float64(5.67983506061483))

In [33]:
meanB = np.mean(groupB)
stdB = np.std(groupB, ddof=1)
meanB, stdB

(np.float64(64.1), np.float64(8.279157230325104))

In [34]:
print(f"A반 평균: {meanA:.2f}, 표준 편차: {stdA:.2f}")
print(f"B반 평균: {meanB:.2f}, 표준 편차: {stdB:.2f}")

A반 평균: 70.55, 표준 편차: 5.68
B반 평균: 64.10, 표준 편차: 8.28


## 정규성 검정(ks-test)

In [45]:
from scipy.stats import * #stats 모듈의 모든 내용을 로딩
# from scipy import stats
import numpy as np

In [58]:
# group_A와 group_B의 평균과 표준편차를 사용하여 정규성 검정 수행
print(kstest(groupA, 'norm', args=(np.mean(groupA), np.std(groupA, ddof=1))))
print(kstest(groupB, 'norm', args=(np.mean(groupB), np.std(groupB, ddof=1))))

KstestResult(statistic=np.float64(0.13310674849448567), pvalue=np.float64(0.8254564220619713), statistic_location=np.int64(73), statistic_sign=np.int8(1))
KstestResult(statistic=np.float64(0.15881009358601705), pvalue=np.float64(0.9295432555006241), statistic_location=np.int64(73), statistic_sign=np.int8(-1))


# 등분산 검정

- 그룹A와 그룹B가 같은 분산 인지 확인하기 위해 levene() 검정 사용

In [57]:
levene(groupA, groupB)

LeveneResult(statistic=np.float64(2.033067087400979), pvalue=np.float64(0.16496408622210104))

In [62]:
np.var(groupA, ddof=1)

np.float64(32.26052631578948)

In [64]:
np.var(groupB, ddof=1)

np.float64(68.54444444444445)

## 독립표본 t-검정 수행

In [69]:
print(ttest_ind(groupA, groupB, equal_var = True)) 

TtestResult(statistic=np.float64(2.5128526794964134), pvalue=np.float64(0.01801095352893767), df=np.float64(28.0))
